In [1]:
%pip install torch tensorboardX tensorboard pandas seaborn


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

Looking in indexes: https://download.pytorch.org/whl/nightly/cpu

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader, SubsetRandomSampler
from tensorboardX import SummaryWriter
import os
from time import perf_counter

In [4]:

class CustomDataSet(Dataset):
    def __init__(self, x : np.ndarray,y : np.ndarray):
        # x: numpy array
        # y: numpy array
        self.data = x
        self.label = y
        
    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index : int):
        data = self.data[index]
        label = self.label[index]
        
        return data, label

In [5]:
class Net(nn.Module):
  def __init__(self,input_shape,layerWidth=512,dropoutRate=0.3):
    """
    Extend of nn.Module
    Wide Neural Network 
    layerWidth: width of the hidden layers
    input_shape: shape of the input
    Binary Classification
    """
    super(Net,self).__init__()
    self.dropoutRate = dropoutRate
    self.fc1 = nn.Linear(input_shape,layerWidth)
    self.fc2 = nn.Linear(layerWidth,layerWidth)
    self.fc3 = nn.Linear(layerWidth,1)
  def forward(self,x):
    # self.training is with nn.Module
    x = torch.relu(self.fc1(x))
    x = torch.dropout(x,self.dropoutRate,train=self.training)
    x = torch.relu(self.fc2(x))
    x = torch.dropout(x,self.dropoutRate,train=self.training)
    x = torch.sigmoid(self.fc3(x))
    return x


In [6]:
# path of CSV file
processedDataDirectoryPath = "processedData"
trainCSVPath = os.path.join(processedDataDirectoryPath,'processed_train_data.csv')
testCSVPath = os.path.join(processedDataDirectoryPath,'processed_test_data.csv')
# read CSV file
# split data and label
label = "smoking"
df = pd.read_csv(trainCSVPath)
y = df[label]
df = df.drop(label, axis=1)
# convert to numpy array
x = df.values
y = y.values
print(x.shape)


(159256, 16)


In [7]:
# Hyperparameters
num_cores = 0
# torch.set_num_threads(num_cores)
layerWidth = 512
lr = 0.05
batchSize = 2048
dropout = 0.3
device = torch.device("mps")
model = Net(x.shape[1],dropoutRate=dropout)
model = model.to(device)
# May try otu different optimizers
optimizer = optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.BCELoss()
validation_split = .2 # 20% of the data is used for validation
dataset = CustomDataSet(x,y)
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
train_indices, val_indices = indices[split:], indices[:split]
# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,sampler=train_sampler,num_workers=num_cores)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batchSize,sampler=valid_sampler,num_workers=num_cores)
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
val_features, val_labels = next(iter(validation_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")


Feature batch shape: torch.Size([2048, 16])
Labels batch shape: torch.Size([2048])
Feature batch shape: torch.Size([2048, 16])
Labels batch shape: torch.Size([2048])


In [8]:
def validationTest(model, val_loader):
    acc = 0
    loss = 0
    model.eval()
    with torch.no_grad():
        for (data, target) in val_loader:
            data = data.to(torch.float32)
            target = target.to(torch.float32)
            data = data.to(device)
            target = target.to(device)
            target = target.unsqueeze(1)
            output = model(data)
            loss = loss_fn(output,target)
            acc += (output.reshape(-1).detach().cpu().numpy().round() == target.reshape(-1).detach().cpu().numpy()).mean()
        loss = loss.item()
    return acc/len(val_loader), loss
    

In [9]:
def train(model,epochs = 10, model_path = "./model.pth",log_path = "log_pytorch"):
   
    tb = SummaryWriter(f"./train_{log_path}/")  # training tensorboard
    vtb = SummaryWriter(f"./val_{log_path}/")  # validation tensorboard

    # put the model into training mode
    model.train()
    outputFormat = "{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|{:10}"
    print(outputFormat.format("Mode","Epoch","Loss","Accuracy","Time Used","Progress"))
    outputFormat = "{:^10}|{:^10.0f}|{:^10.4f}|{:^10.4f}|{:^10.2f}|{:^10}"
    
    for epoch in range(1, epochs + 1):
        startTime = perf_counter()
        length = len(train_loader)
        acc = 0
        loss = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data = data.to(torch.float32)
            target = target.to(torch.float32)
            data = data.to(device)
            target = target.to(device)
            target = target.unsqueeze(1)
            optimizer.zero_grad()
            #calculate output
            output = model(data)
            loss = loss_fn(output,target)
            acc += (output.reshape(-1).detach().cpu().numpy().round() == target.reshape(-1).detach().cpu().numpy()).mean()
            #backprop
            loss.backward()
            optimizer.step()
            print(outputFormat.format("Train",epoch,loss.item(),acc/(batch_idx+1),perf_counter() - startTime,f"{batch_idx+1}/{length}"),end="\r")
        # print(outputFormat.format("Train",epoch,loss.item(),acc/(batch_idx+1),perf_counter() - startTime,f"{batch_idx+1}/{length}"))
        trainAcc = acc/len(train_loader)
        tb.add_scalar("epoch loss", loss.item(), epoch)
        tb.add_scalar("epoch accuracy", trainAcc, epoch)
        print(outputFormat.format("Train",epoch,loss.item(),trainAcc,perf_counter() - startTime,f"{batch_idx+1}/{length}"))

        val_Acc , val_Loss = validationTest(model,validation_loader)
        print(outputFormat.format("Validation",epoch,val_Loss,val_Acc,perf_counter() - startTime,"Done"))
        vtb.add_scalar("epoch loss", val_Loss, epoch)
        vtb.add_scalar("epoch accuracy", val_Acc, epoch)
        for name, weight in model.named_parameters():
            tb.add_histogram(name, weight, epoch)
            tb.add_histogram(f'{name}.grad',weight.grad, epoch)
    # save the model to a .pth file
    print('Saving NN to %s' % model_path)
    torch.save(model.state_dict(), model_path)
    # add graph to tensorboard
    tb.add_graph(model, (data,))

In [10]:
def modelPred(model,test_data,model_path="./model.pth"):

    model.load_state_dict(torch.load(model_path))
    
    # put model into test mode
    model.eval()
    with torch.no_grad():
    
        output = model(torch.tensor(test_data,dtype=torch.float32))
        pred = output.reshape(-1).detach().numpy()
        
        return pred

In [11]:
strLR =  str(lr).replace(".","-")
epoch = 7500
modelStr = f"model_0-1set_{batchSize}b_{strLR}lr_{layerWidth}w_{epoch}e_{dropout}d"
modelPath = f"./{modelStr}.pth"
train(model,epochs=epoch,model_path=modelPath,log_path=modelStr)


   Mode   |  Epoch   |   Loss   | Accuracy |Time Used |Progress  
  Train   |    1     |  0.6775  |  0.5602  |   1.04   |  63/63   
Validation|    1     |  0.6803  |  0.5630  |   1.19   |   Done   
  Train   |    2     |  0.6617  |  0.5826  |   0.86   |  63/63   
Validation|    2     |  0.6702  |  0.6103  |   0.99   |   Done   
  Train   |    3     |  0.6474  |  0.6131  |   0.84   |  63/63   
Validation|    3     |  0.6445  |  0.6166  |   0.98   |   Done   
  Train   |    4     |  0.6428  |  0.6219  |   0.86   |  63/63   
Validation|    4     |  0.6364  |  0.6372  |   1.05   |   Done   
  Train   |    5     |  0.6156  |  0.6451  |   0.84   |  63/63   
Validation|    5     |  0.6102  |  0.6543  |   0.97   |   Done   
  Train   |    6     |  0.5985  |  0.6658  |   0.83   |  63/63   
Validation|    6     |  0.6058  |  0.6704  |   0.97   |   Done   
  Train   |    7     |  0.6065  |  0.6811  |   0.82   |  63/63   
Validation|    7     |  0.5939  |  0.6782  |   0.95   |   Done   
  Train   

In [12]:
testModel = Net(x.shape[1])
test_data = pd.read_csv(testCSVPath)
Id = "id"
y = test_data[Id]
test_data = test_data.drop(Id,axis=1)
predict = modelPred(testModel,test_data=test_data.values,model_path=modelPath)
print(predict)
mapped = {"id":y.values,"smoking" : predict}

df = pd.DataFrame.from_dict(mapped)
df.to_csv("result.csv",index=False)

[0.7671899  0.39921495 0.56766105 ... 0.66326517 0.1958148  0.04868647]
